# Gerador de GRUs em lote da Inova Unicamp

*por Francisco Martellini - setembro 2020*

Este notebook foi construído para uso interno da Agência de Inovação da Unicamp, e possui algumas especificidades para a nomeação dos arquivos gerados e execução no ambiente do Google Colab.

Caso deseje implementá-lo em sua organização, procure pela versão geral (**pyINPI - GRU.ipynb**) no meu repositório do GitHub, que possui o mesmo código sem as especificidades feitas para a Inova Unicamp.

Você pode encontrar meu repositório com o arquivo em **<https://github.com/frmartellini/inpi>**.

## Instruções

### i) Para gerar GRUs em lote e criar o relatório dos boletos

Execute todas as partes deste arquivo, uma de cada vez, no Google Colab. Clique no ícone da pasta na barra lateral e faça o download do pacote zip com todos os arquivos gerados.

### ii) Para gerar uma GRU individual

Execute no Google Colab exatamente nesta ordem: **Parte 1**, **Parte 2**, **Parte 3**, **Parte 5**, **Parte 6**. Clique no ícone da pasta na barra lateral e faça o download do arquivo pdf. **Não execute a Parte 4 e nem as outras partes do programa neste caso!**.

### iii) Para gerar somente o relatório de boletos de GRUs existentes

No Google Colab clique no ícone da pasta da barra lateral e carregue os arquivos PDF das GRUs geradas previamente, e execute exatamente nesta ordem: **Parte 1**, **Parte 2**, **Parte 3** e **Parte 7**. Clique no ícone da pasta na barra lateral e faça o download do arquivo xlsx. Caso não deseje esperar que o Google Colab apague os arquivos da nuvem, depois de fazer o download, execute a **Parte 9**. **Não execute as outras partes do programa neste caso!**

In [ ]:
#@title PARTE 1: CARREGAR BIBLIOTECAS
import requests
import json
from getpass import getpass
import pandas as pd
import numpy as np
import os
import re
from datetime import date, timedelta

# COMENTE ESSAS DUAS LINHAS SE NÂO ESTIVER USANDO NO COLAB!

!pip install -q pdfminer3
!pip install -q XlsxWriter

# COMENTE ESSAS DUAS LINHAS SE NÂO ESTIVER USANDO NO COLAB!

from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import TextConverter
import io
import os
import sys
import sys

print("Ok!")

In [ ]:
#@title PARTE 2: APAGAR ARQUIVOS PRÉ-EXISTENTES

!rm /content/*.pdf # apagando os arquivos pdf
!rm /content/*.xlsx # apagando os arquivos xlsx
!rm /content/*.zip # apagando os arquivos zip

print("Ok!")

In [ ]:
#@title PARTE 3: DEFINIR FUNÇÕES
def create_gru(url, headers, file_name, user, password, cpfcnpj, codigo, guiaanterior, valor, apresentacao, natureza, tipopeticionamento, processo, peticaovinculada, totaldeitens, revista, objetopeticao):
    payload = {
        "login":user,
        "senha":password,
        "cpfCnpjInpiCliente":cpfcnpj,
        "servico": [{
            "codigo":codigo,
            "guiaAnterior":guiaanterior,
            "valor":valor,
            "apresentacao":apresentacao,
            "natureza":natureza,
            "tipoPeticionamento":tipopeticionamento,
            "numeroProcesso":processo,
            "peticaoVinculada":peticaovinculada,
            "total":totaldeitens,
            "revista":revista,
            "objetoPeticao":objetopeticao
        }]
    }
    
    # Enviando a requisição para o sistema de geração de GRUs do INPI
    r = requests.post(url, data=json.dumps(payload), headers=headers, verify=False)
    r.content

    # Fazer o download do arquivo
    folder = "/content" # nome da pasta
    path = folder + "/" + file_name

    with open(path, 'wb') as s:
        s.write(r.content)
        
def parse_txt(text,start,end):
    i = text.find(start) + len(start)
    f = text.find(end)
    txt = text[i:f]
    return txt

class modelo_febraban_BB:
    def __init__(self, nr_codb):
        self.nr_codb = nr_codb
        
    def cod_manual(self): # remove espaços do número de digitação manual de 47 digitos (44 + digitos verificadores dos campos)
        return self.nr_codb.replace('.','').replace(' ','')
    
    def cod_leitora(self): # converte o número de digitação manual no código de barras para a leitora com 44 digitos
        return self.cod_manual()[0:3] + self.cod_manual()[3:4] + self.cod_manual()[32:33] + self.cod_manual()[33:37] + self.cod_manual()[37:47] + self.cod_manual()[4:9] + self.cod_manual()[10:20] + self.cod_manual()[21:31]
    
    def elements(self):
        elements = []
        elements.append(self.cod_manual()[0:3]) # A - numero do banco
        elements.append(self.cod_manual()[3:4]) # B - digito moeda real = 9
        elements.append(self.cod_manual()[4:9]) # C - posições 20 a 24 do código de barras (zeros)
        elements.append(self.cod_manual()[9:10]) # X - digito verificador campo 1
        elements.append(self.cod_manual()[10:20]) # D - Posições 25 a 34 do código de barras (convênio fornecido pelo banco) 
        elements.append(self.cod_manual()[20:21]) # Y - digito verificador campo 2
        elements.append(self.cod_manual()[21:31]) # E - Posições 35 a 44 do código de barras (comp nosso numero/cart mod cobr)
        elements.append(self.cod_manual()[31:32]) # Z - digito verificador campo 3
        elements.append(self.cod_manual()[32:33]) # K - digito verificador do código de barras deve ser diferente de 0
        elements.append(self.cod_manual()[33:37]) # U - Fator de Vencimento (cálculo conforme anexo VI)
        elements.append(self.cod_manual()[37:47]) # V - Valor do boleto de pagamento (com duas casas decimais, sem ponto e vírgula. Em caso de moeda variável, informar zeros)
        return elements
    
    def banco(self): # retorna o banco do boleto
        if self.cod_manual()[0:3] == "001": # GRUs são sempre do BB
            return "Banco do Brasil"
        else: # preciso de uma lista com códigos de banco
            return "Outro banco"
        
    def moeda(self):
        if self.cod_manual()[3:4] == "9":
            return "Real (R$)"
        else: # preciso de uma lista com códigos de banco
            return "Moeda estrangeira"
    
    def vencimento(self): # determina o vencimento pelo fator de vencimento do código de barras
        fator_vencimento = self.cod_manual()[33:37] # fator de vencimento do código de barras
        vencimento = date(2000, 7, 3) + timedelta(days=int(fator_vencimento)-1000) # primeira data 03/07/2000 com fator 1000 e última data 21/02/2025 com fator 9999, e reinicia os fatores de novo 
        return vencimento.strftime('%d/%m/%Y')
    
    def valor(self):
        return int(self.cod_manual()[37:47])/100
    
    def modulo_11(self): # checa o digito verificador do codigo de barras de acordo com o modulo 11
        nr = self.cod_leitora()
        p = len(nr) # posição
        sum_dv = 0
        k = 2 # multiplicador
        for i in range(44):
            if p != 5:
                if 2<=k<=9:
                    sum_dv = sum_dv + int(nr[p-1:p])*k
                if k>9:
                    k = 2
                    sum_dv = sum_dv + int(nr[p-1:p])*k
            if p == 5:
                k-=1
            k+=1
            p-=1
        
        dv = 11 - sum_dv%11
        if dv == 0 or dv == 10 or dv == 11:
            return 1
        if dv != 0 or dv != 10 or dv != 11:
            return dv
    
    def modulo_10(self): # checa o digito dos campos 1, 2 e 3 de acordo com o modulo 10
        i = 0
        j = 9
        dv_f = []
        for x in range(3): # verifica um campo de cada vez
            nr = self.cod_manual()[i:j]
            p = 0
            sum_dv = 0
            for i in range(int(len(nr))): # verifica o campo i
                if i%2 == 1:
                    if len(nr) == 9:
                        k = 1
                    if len(nr) == 10:
                        k = 2
                if i%2 == 0:
                    if len(nr) == 9:
                        k = 2
                    if len(nr) == 10:
                        k = 1
                if (int(nr[p:p+1])*k) > 9:
                    sum_dv = sum_dv + (int(nr[p:p+1])*k)//10 + (int(nr[p:p+1])*k)%10
                if (int(nr[p:p+1])*k) <= 9:
                    sum_dv = sum_dv + int(nr[p:p+1])*k
                p+=1
            
            dv = 10 - sum_dv%10
            if dv == 10:
                dv_f.append(0)
            if dv != 10:
                dv_f.append(dv)
            i = j + 1
            j = i + 10
            
        return dv_f
    
class pdf_parse:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
            
    def pdf_to_txt(self): # source: https://stackoverflow.com/questions/56494070/how-to-use-pdfminer-six-with-python-3
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)

        with open(self.pdf_path, 'rb') as fh:
            for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
                page_interpreter.process_page(page)

            text = fake_file_handle.getvalue()

        # close open handles
        converter.close()
        fake_file_handle.close()

        return text

def excel_report(xls_df,xls_name,sheet_name):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    # also set the default datetime and date formats.
    writer = pd.ExcelWriter(xls_name + ".xlsx", engine='xlsxwriter', date_format='dd/mm/yyyy')
    # Convert the dataframe to an XlsxWriter Excel object.
    xls_df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Get the xlsxwriter workbook and worksheet objects in order to set the column
    # widths, to make the dates clearer.
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    
    # Set the previous defined sum cell in the spreadsheet
    worksheet.write(row, 3, soma)
    
    # Add a number format for cells with money.
    money_fmt = workbook.add_format({'align': 'center', 'num_format': 'R$ #.##0'})
    align_fmt = workbook.add_format({'align': 'center'})
    worksheet.set_column('A:A', 30)
    worksheet.set_column('B:B', 20, align_fmt)
    worksheet.set_column('C:C', 25, align_fmt)
    worksheet.set_column('D:D', 15, money_fmt)
    worksheet.set_column('E:E', 15, align_fmt)
    worksheet.set_column('F:F', 80)
       
    # Close the Pandas Excel writer and output the Excel file
    writer.save()
    print("Relatório concluído.")
        
# Para uso no google colab
colab = False
print("Ok!")

In [ ]:
#@title PARTE 4: CARREGAR PLANILHA COM A LISTA DE GRUs

# Para uso no Google Colab

from google.colab import files # para uso no google colab
!pip install -q XlsxWriter # para uso no google colab

print("Carregue a planilha com lista de GRUs\n")
uploaded = files.upload()

for fn in uploaded.keys():
    arquivo=fn
    colab = True
    
print("\nOk!")

In [ ]:
#@title PARTE 5: INFORMAÇÔES DE LOGIN NO INPI
# Login
# O mesmo utilizado para acesso à geração de GRU, peticionamentos eletrônicos e busca
user = getpass('Digite o nome de usuário: ')

# Senha
# A mesma utilizada para acesso à geração de GRU, peticionamentos eletrônicos e busca
password = getpass('Digite a senha: ')

# CPF ou CNPJ do cliente
# Número do CPF/CNPJ do requisitante ou número INPI, no caso de clientes estrangeiros
cpfcnpj = input("Digite o CPF ou o CNPJ do cliente sem hífen \n [ENTER para o CNPJ da UNICAMP]: ") or "46068425000133"

print("\nOk!")

In [ ]:
#@title PARTE 6: GERAR GRUs
# Origem do arquivo com a lista de GRUs
em_lote = False
if colab == True:
    f_gru = arquivo
    em_lote = True
if colab == False and os.path.isfile('lista_de_grus.xlsx') == True:
    f_gru = "lista_de_grus.xlsx"
    em_lote = True
    
# Definindo a URL para o método post, o cabeçalho JSON e o payload
url = "https://gru.inpi.gov.br/pag/gru"
headers = {'content-type': 'application/json'}

if em_lote == True:
    gru_df = pd.read_excel(f_gru, sheet_name='Sheet1',converters={'Tecnologia':str,'Despacho':str,'Descrição':str,'Data':str,'Processo':str,'Serviço':str})
    gru_df.columns = map(str.lower, gru_df.columns) # coloca o nome das colunas em minúsculo
    
    ### PADRÂO DE FORMATAÇÃO DE NOMES DA INOVA
    
    # Caso das tecnologias antigas em que o nome da tecnologia deve ser vazio
    # Se o valor da coluna tecnologia é igual a processo (True), troca tecnologia por ""
    # Se o valor da coluna tecnologia é diferente de processo (False), mantêm os valores da coluna tecnologia
    gru_df['tecnologia'] = np.where(gru_df['tecnologia'] == gru_df['processo'], "", gru_df['tecnologia'])
    
    # Padrão de nome para as datas
    gru_df['data'] = gru_df['data'].str[0:2] + gru_df['data'].str[3:5] + gru_df['data'].str[8:10]
    
    # Nome dos arquivos das GRUs - Padrão para a RPI da semana
    gru_df['file_name'] = gru_df['tecnologia'] + "_" + gru_df['processo'] + "_GRU_" + gru_df['despacho'] + "_" + gru_df['descricao'] + "_" + gru_df['data'] + ".pdf"
    
    # Correção no nome do arquivo no caso de tecnologia antiga, o código anterior coloca underline no começo do nome
    gru_df['file_name'] = np.where(gru_df['file_name'].str[0:1] == "_", gru_df['file_name'].str[1:], gru_df['file_name'])
    
    ### PADRÂO DE FORMATAÇÃO DE NOMES DA INOVA
    
    # Troca todos os valores NaN por None
    gru_df = gru_df.replace({np.nan: None})
    
    # Envia a requisição e faz o download das GRUs
    for file_name, codigo, guiaanterior, valor, apresentacao, natureza, tipopeticionamento, processo, peticaovinculada, totaldeitens, revista, objetopeticao in zip(gru_df['file_name'],gru_df['servico'],gru_df['guiaanterior'],gru_df['valor'],gru_df['apresentacao'],gru_df['natureza'],gru_df['tipopeticionamento'],gru_df['processo'],gru_df['peticaovinculada'],gru_df['totaldeitens'],gru_df['revista'],gru_df['objetopeticao']):
        create_gru(url, headers, file_name, user, password, cpfcnpj, codigo, guiaanterior, valor, apresentacao, natureza, tipopeticionamento, processo, peticaovinculada, totaldeitens, revista, objetopeticao)
    
if em_lote == False:
    ### PADRÂO DE FORMATAÇÃO DE NOMES DA INOVA
    
    tecnologia = input("Digite o nome da tecnologia ou deixe vazio se ela for no padrão antigo: ")
    despacho = input("Digite o número do despacho: ")
    descricao = input("Digite a descrição do arquivo: ")
    data = input("Digite a data: ")
    
    ### PADRÂO DE FORMATAÇÃO DE NOMES DA INOVA
    
    # Entrada de valores para o serviço desejado
    
    # Código do serviço
    codigo = input("Digite o código do serviço [por enquanto só o 207, 212, 281!]: ")
    codigo = int(codigo)
    
    # Número da GRU anterior no caso de serviços que exijam o preenchimento do número de GRU paga anteriormente,
    # como complementação de retribuição
    guiaanterior = None
    
    # Valor da GRU para serviços sem valor determinado, como complementação de retribuição
    valor = None
    
    # Código numérico da apresentação para o caso de depósito de Marcas, sendo:
    # 1 - Nominativa, 2 - Figurativa, 3 - Mista, 4 - Tridimensional
    apresentacao = None
    
    # Código numérico da natureza para os serviços que exigem esta informação,
    # como os depósitos de Marcas, Patentes e DI, sendo:
    # i) Desenho Industrial:
    #    1 - Depósito de Pedidos de Registro de Desenho Industrial (DI), 2 - Pedidos Divididos
    # ii) Patentes:
    #     1 - Patente de Invenção (PI), 2 - Certificado de Adição (C), 3 - Modelo de Utilidade (MU),
    #     4 - Patente de Invenção (PI) via PCT, 5 - Patente de Invenção (PI) dividido,
    #     6 - Modelo de Utilidade (MU) via PCT, 7 - Modelo de Utilidade (MU) dividido
    # iii) Marcas:
    #      1 - Produto, 2 - Serviço, 3 - Coletiva, 4 - Certificação
    natureza = None
    
    # Tipo de Peticionamento
    # Código numérico para os serviços que exigem esta informação, sendo:
    # 0 - Papel, 1 - Eletrônico
    tipopeticionamento = None
    
    # Número de processo para as petições relacionadas a processos
    # teste - BR 10 2020 009949-3
    processo = input("Digite o número do processo: ") or None
    
    # Número da petição vinculada para os serviços relacionados a petições anteriores
    peticaovinculada = None
    
    # Total de itens para serviços que exigem esta informação, como os pedidos de averbação de contrato
    totaldeitens = None
    
    # Número da RPI para serviços que exigem esta informação, como os pedidos de retificação por erro
    # de publicação na RPI
    revista = None
    
    # Código numérico do tipo de objeto da petição para os serviços que exigem esta informação
    objetopeticao = None
    
    ### PADRÂO DE FORMATAÇÃO DE NOMES DA INOVA
    
    file_name = tecnologia + "_" + processo + "_GRU_" + despacho + "_" + descricao + "_" + data + ".pdf"
    
    # Correção do nome para o caso das tecnologias antigas
    if file_name.startswith('_'):
        file_name = file_name[1:]
        
    ### PADRÂO DE FORMATAÇÃO DE NOMES DA INOVA
    
    # Envia a requisição e faz o download da GRU
    create_gru(url, headers, file_name, user, password, cpfcnpj, codigo, guiaanterior, valor, apresentacao, natureza, tipopeticionamento, processo, peticaovinculada, totaldeitens, revista, objetopeticao)

del user
del password

print("Ok!")

In [ ]:
#@title PARTE 7: CRIAR O RELATÓRIO DE GRUs
# Gera o relatório de GRUs a partir dos PDFs

files_list = []
with os.scandir() as files:
    for entry in files:
        if entry.name.endswith('.pdf') and entry.is_file():
            files_list.append(entry.name)
            
data = files_list[0].split("_")[-1].split(".")[0]

row = 0
report_df = pd.DataFrame(columns=['Tecnologia', 'PI', 'Número', 'Valor','Vencimento','Serviço'])
for i in range(len(files_list)):
    file = files_list[i]
    #print(file) # imprimir o nome do arquivo
    text = pdf_parse(file).pdf_to_txt()
            
    # extrair codigo de barra
    scan_cod = parse_txt(text,"Pessoa Jurídica.\n\n","\n\nUNIVERSIDADE")
    
    # verifica se o código de barra foi extraído corretamente
    if scan_cod == "": # se não foi, imprime uma mensagem de erro no relatório
        check_cb = False
        report_df.loc[i] = [file,"Erro! Verifique o arquivo!","","","",""]
        row=i
    if scan_cod != "":
        check_cb = True
        
    if check_cb == True:
        # se o código foi extraído corretamente, checa o código de barra
        # com base nas validações do modulo_10 e do módulo_11
        scan_cod2 = modelo_febraban_BB(scan_cod).cod_manual()
        flag1 = int(scan_cod2[9:10]) == modelo_febraban_BB(scan_cod).modulo_10()[0]
        flag2 = int(scan_cod2[20:21]) == modelo_febraban_BB(scan_cod).modulo_10()[1]
        flag3 = int(scan_cod2[31:32]) == modelo_febraban_BB(scan_cod).modulo_10()[2]
        flag4 = int(scan_cod2[32:33]) == modelo_febraban_BB(scan_cod).modulo_11()
        
        # se o código de barra passar nos testes do módulo 10 e do módulo 11,
        # extrai as informações
        if flag1 == flag2 == flag3 == flag4 == True:            
            ## Tecnologia
            if file.split("_")[0].isnumeric() == True:
                tecnologia = file.split("_")[0] + "_" + file.split("_")[1]
            if file.split("_")[0].isnumeric() == False:
                tecnologia = file.split("_")[0]
             
            ## Número do processo
            if text.find("Processo") != -1:
                processo = parse_txt(text,"\nProcesso: "," \nServiço").replace('\n','')
            if processo[0].isdigit() == True:
                processo = "BR " + processo[:2] + " " + processo[2:6] + " " + processo[6:12] + " " + processo[12:13]
            
            ## Valor da GRU
            valor = modelo_febraban_BB(scan_cod2).valor()
            
            ## Número da nota
            # deve ser extraído do código da leitora com 11 dígitos. Ex.:
            #     modelo_febraban_BB("00190.00009 02940.916196 22483.221176 1 83750000023600").cod_leitora()
            #   > 00191837500000236000000002940916192248322117
            #   >                      Nr: 29409161922483221
            nr_nota = modelo_febraban_BB(scan_cod2).cod_leitora()[25:42]
            
            ## Serviço
            if text.find("Serviço") != -1:
                servico = parse_txt(text," \nServiço: "," \n\nUNIVERSIDADE").replace('\n','')
            if text.find("Serviço") == -1:
                servico = ""
            
            # Vencimento
            vencimento = modelo_febraban_BB(scan_cod2).vencimento() # extrai o vencimento
            
            # Adiciona as informações ao dataframe
            report_df.loc[i] = [tecnologia,processo,nr_nota,valor,vencimento,servico]
        
        # se não passar, imprime uma mensagem de erro no relatório
        if flag1 == flag2 == flag3 == flag4 == False:            
            report_df.loc[i] = [file,"Erro! Verifique o arquivo!","","","",""]
            row=i
            
        row = i

row+=2        
soma = "=SUM(D2:D" + str(row) + ")"

# Configurando o arquivo em excel
f_name = "CARTA_DGA_" + str(data) + "_" + "Anexo" # nome do arquivo sem a extensão xlsx
s_name = "CARTA DGA " + str(data)
excel_report(report_df,f_name,s_name)

In [ ]:
#@title PARTE 8: CRIAR O ARQUIVO ZIP

# Para uso no Google Colab
!rm "$arquivo"

# download dos arquivos em zip
zip_file = "INPI_GRUs.zip"

!zip -r "$zip_file" *.pdf *.xlsx

print("Ok!")

In [ ]:
#@title PARTE 9: LIMPAR ARQUIVOS DA NUVEM
!rm /content/*.pdf # apagando os arquivos pdf
!rm /content/*.xlsx # apagando os arquivos xlsx

print("Ok!")